# Test Executor 

In [14]:
from ragas.executor import Executor
from asyncio import sleep

exec = Executor(raise_exceptions=True)
for i in range(10):
    exec.submit(sleep, i / 10)

assert exec.results(), "didn't get anything from results"

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
from ragas.executor import is_event_loop_running

assert is_event_loop_running() is True, "is_event_loop_running() returned False"

In [16]:
from ragas.executor import as_completed
import asyncio


async def echo_order(index: int):
    await asyncio.sleep(index)
    return index


async def _run():
    results = []
    for t in as_completed([echo_order(1), echo_order(2), echo_order(3)], 3):
        r = await t
        results.append(r)
    return results


results = await _run()

expected = [1, 2, 3]
assert results == expected, f"got: {results}, expected: {expected}"

In [17]:
exec = Executor(raise_exceptions=True)
for i in range(1000):
    exec.submit(sleep, 0.01)

assert exec.results(), "didn't get anything from results"

for i in range(1000):
    exec.submit(sleep, 0.01)

assert exec.results(), "didn't get anything from results"

Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2000 [00:00<?, ?it/s]

# Test Metric

In [23]:
from ragas.metrics.base import Metric


class FakeMetric(Metric):
    name = "fake_metric"
    _required_columns = ("user_input", "response")

    def init(self):
        pass

    async def _ascore(self, row, callbacks) -> float:
        return 0


fm = FakeMetric()

In [24]:
score = fm.score({"user_input": "a", "response": "b"})
assert score == 0